# Graph Neural Network (from Scratch)

## Graph Convolutional Network (GCN)

In [ ]:
X = [.25, .5, .75, 1]
# X = [1, 1, 1, 1]
X = torch.tensor(X, dtype=torch.float)

weight = [.1, .2, .3, .4]
# weight = [1, 1, 1, 1]
weight = torch.tensor(weight, dtype=torch.float)

A_square = [
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]
]

A_disconnected = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

A_half_triangle = [
    [0, 0, 1, 1],
    [0, 0, 0, 0],
    [1, 0, 0, 1],
    [1, 0, 1, 0]
]

for lbl, A in zip(
    ["A_square", "A_disconnected", "A_half_triangle"],
    [A_square, A_disconnected, A_half_triangle]
):
    A = torch.tensor(A)
    A_hat = A + torch.eye(A.size(0))
    D = torch.diag(A_hat.sum(dim=1))

    D_inv_sqrt = torch.pow(D, -0.5)
    D_inv_sqrt[torch.isinf(D_inv_sqrt)] = 0.
    A_norm = D_inv_sqrt @ A_hat @ D_inv_sqrt

    graph_by_input = A_norm @ X
    z = graph_by_input @ weight

    out = torch.relu(z)

    print(lbl, "\n", A_hat, "\n", D, "\n", D_inv_sqrt, "\n", A_norm, "\n", X, "\n", graph_by_input, "\n", z, "\n", out)

A_square 
 tensor([[1., 1., 0., 1.],
        [1., 1., 1., 0.],
        [0., 1., 1., 1.],
        [1., 0., 1., 1.]]) 
 tensor([[3., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 3.]]) 
 tensor([[0.5774, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5774, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5774, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5774]]) 
 tensor([[0.3333, 0.3333, 0.0000, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.0000, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.0000, 0.3333, 0.3333]]) 
 tensor([0.2500, 0.5000, 0.7500, 1.0000]) 
 tensor([0.5833, 0.5000, 0.7500, 0.6667]) 
 tensor(0.6500) 
 tensor(0.6500)
A_disconnected 
 tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]) 
 tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]) 
 tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
      

In [2]:
import torch
import torch.nn as nn

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
    
    def forward(self, X, A):
        """
        X: Node features (N x in_features)
        A: Adjacency matrix (N x N)
        """
        # Add self-loops
        A_hat = A + torch.eye(A.size(0)) # identity matrix of size adjacency matrix
        
        # Compute degree matrix
        D = torch.diag(A_hat.sum(dim=1))
        
        # Symmetric normalization: D^(-1/2) A D^(-1/2)
        D_inv_sqrt = torch.pow(D, -0.5)
        D_inv_sqrt[torch.isinf(D_inv_sqrt)] = 0.
        A_norm = D_inv_sqrt @ A_hat @ D_inv_sqrt
        
        # Aggregate and transform
        H = A_norm @ X @ self.weight
        
        return torch.relu(H)

# Full GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.layer1 = GCNLayer(input_dim, hidden_dim)
        self.layer2 = GCNLayer(hidden_dim, output_dim)
    
    def forward(self, X, A):
        H = self.layer1(X, A)
        H = self.layer2(H, A)
        return H